# Decorators continuation

https://realpython.com/primer-on-python-decorators/#fancy-decorators

## Fancy Decorators

### Decorating Classes

Some commonly used decorators that are even built-ins in Python 
are `@classmethod`, `@staticmethod`, and `@property`.  

* The @classmethod and @staticmethod decorators are used to define 
    methods inside a class namespace that are not connected to a particular 
    instance of that class. 
* The @property decorator is used to customize getters and setters 
    for class attributes.

In [2]:
import functools

In [3]:
class Circle:
    def __init__(self, radius):
        self._radius = radius

    @property
    def radius(self):
        """Get value of radius"""
        return self._radius

    @radius.setter
    def radius(self, value):
        """Set radius, raise error if negative"""
        if value >= 0:
            self._radius = value
        else:
            raise ValueError("Radius must be positive")

    @property
    def area(self):
        """Calculate area inside circle"""
        return self.pi() * self.radius**2

    @property
    def area_times_n(self):
        """Calculate area inside circle"""
        return self.pi() * self.radius**2 * self.n

    def cylinder_volume(self, height):
        """Calculate volume of cylinder with circle as base"""
        return self.area * height

    @classmethod
    def unit_circle(cls):
        """Factory method creating a circle with radius 1 & n = 1"""
        return cls(1)

    @staticmethod
    def pi():
        """Value of π, could use math.pi instead though"""
        return 3.1415926535

In [4]:
c = Circle(5)
c.n = 10
print(c._radius)
print(c.radius)
print(c.area)
print(c.area_times_n)
c.cv =  c.cylinder_volume(height=c.n)
print(c.cv)
print(c.pi())
print(Circle.pi())
c = Circle.unit_circle()
print('New radius:', c.radius)


5
5
78.5398163375
785.398163375
785.398163375
3.1415926535
3.1415926535
New radius: 1


**Example: Use decorators from earlier inside a class**

In [5]:
from decorators import debug, timer

class TimeWaster:
    @debug
    def __init__(self, max_num):
        self.max_num = max_num

    @timer
    def waste_time(self, num_times):
        for _ in range(num_times):
            sum([i**2 for i in range(self.max_num)])

tw = TimeWaster(1000)
tw.waste_time(999)

Calling __init__(<__main__.TimeWaster object at 0x7fb23826aa00>, 1000)
'__init__' returned None
Finished 'waste_time' in 0.1737 secs


**Example: The whole class is decorated**

In [6]:
from decorators import timer

# timer applied in the whole class
@timer
class TimeWaster:
    def __init__(self, max_num):
        self.max_num = max_num

    def waste_time(self, num_times):
        for _ in range(num_times):
            sum([i**2 for i in range(self.max_num)])
        
tw = TimeWaster(1000)
tw.waste_time(999)


Finished 'TimeWaster' in 0.0000 secs


### Nesting decorators

In [7]:
from decorators import debug, do_twice

# The order of @'s matters
@debug
@do_twice
def greet(name):
    print(f"Hello {name}")

greet("Eva")

Calling greet('Eva')
Hello Eva
Hello Eva
'greet' returned None


### Decorators with arguments

In [8]:
# Pass an argument to a decorator
def repeat(num_times):
    def decorator_repeat(func):
        @functools.wraps(func)
        def wrapper_repeat(*args, **kwargs):
            for _ in range(num_times):
                value = func(*args, **kwargs)
            return value
        return wrapper_repeat
    return decorator_repeat

@repeat(num_times=4)
def greet(name):
    print(f"Hello {name}")

greet("Kawabunga")

Hello Kawabunga
Hello Kawabunga
Hello Kawabunga
Hello Kawabunga


### Stateful decorators

**Example: A decorator that counts the number of times a function is called.**  

The state—the number of calls to the function—is stored in the function 
attribute .num_calls on the wrapper function.

In [10]:
def count_calls(func):
    @functools.wraps(func)
    def wrapper_count_calls(*args, **kwargs):
        wrapper_count_calls.num_calls += 1
        print(f"Call {wrapper_count_calls.num_calls} of {func.__name__!r}")
        return func(*args, **kwargs)
    wrapper_count_calls.num_calls = 0
    return wrapper_count_calls

@count_calls
def say_whee():
    print("Whee!")

say_whee()
say_whee()
say_whee.num_calls

Call 1 of 'say_whee'
Whee!
Call 2 of 'say_whee'
Whee!


2

#### Classes as Decorators

**Rewrite the @count_calls example using a class as a decorator**

In [13]:
class CountCalls:
    def __init__(self, func):
        functools.update_wrapper(self, func)
        self.func = func
        self.num_calls = 0

    def __call__(self, *args, **kwargs):
        self.num_calls += 1
        print(f"Call {self.num_calls} of {self.func.__name__!r}")
        return self.func(*args, **kwargs)

@CountCalls
def say_whee():
    print("Whee!")

say_whee()
say_whee()
say_whee.num_calls

Call 1 of 'say_whee'
Whee!
Call 2 of 'say_whee'
Whee!


2

#### Creating Singletons

#### Caching Return Values

#### Adding Information About Units

#### Validating JSON
